## This is a debuging ipython notebook

In [1]:
%load_ext autoreload
import argparse
import warnings
warnings.filterwarnings('ignore')

arg_parser = argparse.ArgumentParser()
        
    # # Add standard arguments
    # if not is_standalone:
    #     # If standalone, set in parser.
arg_parser.add_argument('--output_path', default='.',
                                help='[Default is current directory] Path to '
                                     'output directory.')
arg_parser.add_argument('--version', action='version',
                            help='Show programme\'s version number and exit.',
                            version='0.1')

arg_parser.add_argument('--standalone_path', default='/Users/daviddylus/Research/pore2tree/reference_datasets/Dataset1/',
                            help='[Default is current directory] Path to '
                                 'oma standalone directory.', required=True)
arg_parser.add_argument('--remove_species', default=None,
                            help='Remove species present in dataset and only do analysis on '
                                 'subset of species')


arg_parser.add_argument('--ref_og_aa_folder', default='.',
                            help='Path to preselected og_aa folder')

arg_parser.add_argument('--single_mapping', default=None,
                            help='Single species file allowing to map in a job array.')

arg_parser.add_argument('--threads', type=int, default=None,
                            help='Number of threads for the mapping using ngm / ngmlr!')



    # Arguments to generate the reference
arg_parser.add_argument('-r', '--reference', action='store_true',
                            help='Just generate the reference dataset for mapping')
arg_parser.add_argument('--min_species', type=int, default=30,
                            help='Min number of species in selected orthologous groups. \
                            If not selected it will be estimated such that around 1000 OGs are available.')
arg_parser.add_argument('--dna_reference', default='/Volumes/Untitled/eukaryotes.cdna.fa',
                            help='Reference fasta file that contains nucleotide sequences.')

    # Arguments to map the reads
arg_parser.add_argument('--ref_folder', default=None,
                            help='Folder containing reference files with sequences sorted by species.')
arg_parser.add_argument('--reads', default='/Users/daviddylus/Research/pore2tree/fritz_scripts/pipeline/Nano_reads/ERR1877969.fastq',
                            help='Reads to be mapped to reference.')
# arg_parser.add_argument('--reads', nargs='2',default=None, help='Reads to be mapped to reference.')


# Parse the arguments.
# reference = ['--ref_folder', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test1/db/']
argv = ['--output_path','/Volumes/Untitled/test_run4/','--standalone_path', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/oma_standalone/', '--min_species', 
        '5', '--dna_reference', '/Users/daviddylus/Downloads/eukaryotes.cdna.fa',
        '--reads', '/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test3/test_1.fq /Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test3/test_2.fq']
args = arg_parser.parse_args(argv)
print(args)

Namespace(dna_reference='/Users/daviddylus/Downloads/eukaryotes.cdna.fa', min_species=5, output_path='/Volumes/Untitled/test_run4/', reads='/Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test3/test_1.fq /Users/daviddylus/Research/pore2tree/pore2tree/tests/mapper/test3/test_2.fq', ref_folder=None, ref_og_aa_folder='.', reference=False, remove_species=None, single_mapping=None, standalone_path='/Users/daviddylus/Research/pore2tree/pore2tree/tests/oma_standalone/', threads=None)


In [2]:
from pore2tree.parser import OMAOutputParser
oma_output = OMAOutputParser(args)
args.oma_output_path = oma_output.oma_output_path

Loading files for pre-filter: 100%|██████████| 6428/6428 [00:13<00:00, 492.78 OGs/s]


In [3]:
from pore2tree.Progress import Progress
progress = Progress(args, oma_output)
print(progress.status)

2


In [4]:
from pore2tree.OGSet import OGSet
if progress.status >= 1:
    ogset = OGSet(args, oma_output, load=False)
else:
    ogset = OGSet(args, oma_output)

Loading files: 4 OGs [00:00, 35.30 OGs/s]

--- Re-load ogs and find their corresponding DNA seq from output folder ---


Loading files: 4111 OGs [00:57, 71.71 OGs/s] 


In [5]:
from pore2tree.ReferenceSet import ReferenceSet
if progress.status >= 2:
    reference = ReferenceSet(args, load=False)
else:
    reference = ReferenceSet(args, og_set=ogset.ogs, load=True)

Loading references for mapping from folder:  14%|█▍        | 1/7 [00:00<00:00,  7.12 species/s]

--- Generating reference for mapping from folder ---


Loading references for mapping from folder: 100%|██████████| 7/7 [00:01<00:00,  5.17 species/s]


In [6]:
# %load_ext autoreload
#%autoreload
from pore2tree.Mapper import Mapper
if progress.status >= 3:
    mapper = Mapper(args, og_set=ogset.ogs, load=False)
else:
    mapper = Mapper(args, ref_set=reference.ref, og_set=ogset.ogs)

Mapping reads to species:   0%|          | 0/7 [00:00<?, ? species/s]

--- Mapping of reads to reference sequences ---


Mapping reads to species: 100%|██████████| 7/7 [05:16<00:00, 43.58s/ species]


In [8]:
new_ogs = ogset.add_mapped_seq(mapper.og_records)

In [ ]:
%reload_ext autoreload 
%autoreload 2 
from pore2tree import Aligner
alignments = Aligner.Aligner(args, ogset.mapped_ogs)

Aligning OGs :   0%|          | 0/1909 [00:00<?, ? OG/s]

--- Alignment of OGs ---


Aligning OGs :  37%|███▋      | 705/1909 [05:49<12:22,  1.62 OG/s]

In [ ]:
concat_alignment = alignments.concat_alignment()

In [ ]:
from pore2tree import TreeInference
tree = TreeInference.TreeInference(args, concat_alignment=concat_alignment)

In [ ]:
print(tree.tree)

In [ ]:
import re
bla = 'bascdd [KADO]'
re.search('^(.*?)\[(.*)\]', '', bla)

In [ ]:
import pyopa
